In [0]:
import numpy as np
import cv2
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
m=sum([len(files) for r,d, files in os.walk("/content/gdrive/My Drive/HumanActionClassification")])
X = np.zeros((m,224,224,3))
Y = np.zeros((m, 7))
count = 0
for i in range(1, 8):
    #print(content/gdrive/My Drive/HumanActionClassification/1")
    for file in os.listdir("/content/gdrive/My Drive/HumanActionClassification/" +str(i)+"/"):
        X[count] = cv2.resize(cv2.imread("/content/gdrive/My Drive/HumanActionClassification/"+str(i)+"/"+file), dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        Y[count][i-1] = 1
        count += 1
        print(count)
X = X[:count]
Y = Y[:count]
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.25, random_state = 0)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
def cnn(learning_rate, epochs, batch_size, xTrain, yTrain, xTest, yTest):
    x = tf.placeholder(tf.float32, [None, X.shape[1], X.shape[2], X.shape[3]])
    y = tf.placeholder(tf.float32, [None, Y.shape[1]])
    first_layer = tf.nn.relu(tf.nn.conv2d(x , tf.Variable(tf.truncated_normal([5, 5, X.shape[3], 16], stddev = 0.05), name='W1'), [1, 1, 1, 1], padding='SAME') + tf.Variable(tf.truncated_normal([16]), name='b1'))
    pooled_first_layer = tf.nn.max_pool(first_layer, ksize=[1, 3, 3, 1], strides=[1, 4, 4, 1] , padding='SAME')
    second_layer = tf.nn.relu(tf.nn.conv2d(pooled_first_layer , tf.Variable(tf.truncated_normal([5, 5, 16, 32], stddev = 0.05), name='W2'), [1, 1, 1, 1], padding='SAME') + tf.Variable(tf.truncated_normal([32]), name='b2'))
    pooled_second_layer = tf.nn.max_pool(second_layer, ksize=[1, 3, 3, 1], strides=[1, 4, 4, 1], padding='SAME')
    third_layer = tf.nn.relu(tf.nn.conv2d(pooled_second_layer , tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev = 0.05), name='W3'), [1, 1, 1, 1], padding='SAME') + tf.Variable(tf.truncated_normal([64]), name='b3'))
    pooled_third_layer = tf.nn.max_pool(third_layer, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    flat_layer = tf.reshape(pooled_third_layer, [-1, 7*7*64])
    output = tf.matmul(flat_layer, tf.Variable(tf.truncated_normal([7*7*64, 7], stddev = 0.05), name='W4')) + tf.Variable(tf.truncated_normal([7]), name='b4')        
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels = y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(tf.nn.softmax(output), 1)), tf.float32))
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        total_batch = xTrain.shape[0]//batch_size
        for epoch in range(epochs):
            avg_cost = 0
            for i in range(total_batch):
                _, c = sess.run([optimizer, cross_entropy_loss], feed_dict={x:xTrain[i*batch_size:(i+1)*batch_size], y:yTrain[i*batch_size:(i+1)*batch_size]})
                avg_cost += c/total_batch
            test_accuracy = sess.run(accuracy, feed_dict={x:xTest, y:yTest})
            print("Epoch:", (epoch+1))
            print("Cost:", avg_cost)
            print("Test accuracy:", test_accuracy)
        print("Training Done")
        print("Training accuracy:", sess.run(accuracy, feed_dict = {x:xTrain, y:yTrain})*100, "%")
        print("Testing accuracy:", sess.run(accuracy, feed_dict = {x:xTest, y:yTest})*100, "%")

In [8]:
cnn(0.0000998754, 50, 1, xTrain, yTrain, xTest, yTest)

Instructions for updating:
Colocations handled automatically by placer.
Epoch: 1
Cost: 13.587514288396333
Test accuracy: 0.2631579
Epoch: 2
Cost: 4.350517596998963
Test accuracy: 0.24561404
Epoch: 3
Cost: 2.5386025273321096
Test accuracy: 0.25438598
Epoch: 4
Cost: 1.619259094582993
Test accuracy: 0.25877193
Epoch: 5
Cost: 1.0690106624244338
Test accuracy: 0.25877193
Epoch: 6
Cost: 0.7416069020749192
Test accuracy: 0.2850877
Epoch: 7
Cost: 0.4128080289035776
Test accuracy: 0.28070176
Epoch: 8
Cost: 0.3221169582793825
Test accuracy: 0.27631578
Epoch: 9
Cost: 0.26423258611972167
Test accuracy: 0.25
Epoch: 10
Cost: 0.2780150235274144
Test accuracy: 0.27192983
Epoch: 11
Cost: 0.24896445781638274
Test accuracy: 0.25438598
Epoch: 12
Cost: 0.22369662876728796
Test accuracy: 0.27192983
Epoch: 13
Cost: 0.062326476456083926
Test accuracy: 0.23684211
Epoch: 14
Cost: 0.036193354184908044
Test accuracy: 0.21491228
Epoch: 15
Cost: 0.41533807779812615
Test accuracy: 0.26754385
Epoch: 16
Cost: 0.044920